In [1]:
words = open('names.txt', 'r').read().splitlines()

In [18]:
# Since we also need to take into accoutn when the word is starting and ending, we include '.'
chars = set(''.join(x for x in words))
chars = sorted(list(set(''.join(x for x in words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}


In [19]:
import torch

In [20]:
xs, ys = [], []

for w in words:
    w = list('.') + list(w) + list('.')
    for char1, char2 in zip(w, w[1:]):
        xs.append(stoi[char1])
        ys.append(stoi[char2])
        
xs= torch.tensor(xs)    
ys = torch.tensor(ys)
num = xs.nelement()

In [21]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad = True)

In [22]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float()

for k in range(100):
    logits = xenc @ W
    count = logits.exp()
    prob = count/count.sum(1, keepdims=True)
    loss = -prob[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad
print(loss.item())    

2.4901304244995117


In [24]:
# Sampling from the neural net model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes = 27).float()
        logits = xenc @ W
        count = logits.exp()
        prob = count/count.sum(1, keepdims=True)
        
        ix = torch.multinomial(prob, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        
        if ix == 0:
            break
            
    print(''.join(out))

junide.
janasah.
p.
cfay.
a.
